<a href="https://colab.research.google.com/github/rivkms/2019_omoc_ai/blob/main/Dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm

In [ ]:
!unzip -qq '/content/open.zip' -d '/content/'

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train = pd.read_csv('./train/train.csv')

# 1. LSTM 코드



In [ ]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)

In [ ]:
def train_lstm(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        # X_train = torch.tensor(X_train).float().to(DEVICE)
        # y_train = torch.tensor(y_train).float().to(DEVICE)
        X_train = torch.tensor(np.array(X_train)).float().to(DEVICE)
        y_train = torch.tensor(np.array(y_train)).float().to(DEVICE)


        model = MultiOutputLSTM(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 1)
        }

    return trained_models

In [ ]:
# 학습
trained_models = train_lstm(train)

Training LSTM: 100%|██████████| 193/193 [12:39<00:00,  3.94s/it]


In [ ]:
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals.reshape(-1, 1))
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)

In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('./test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/tmp/ipython-input-165432389.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x_input = torch.tensor([recent_vals]).float().to(DEVICE)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: Use

# 2. 1-D CNN

In [ ]:
class MultiOutputCNN(nn.Module):
    def __init__(self, input_dim=1, output_dim=7, num_filters=64, kernel_size=3):
        super(MultiOutputCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=num_filters, kernel_size=kernel_size)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(num_filters, output_dim)

    def forward(self, x):
        # x: (B, T, C) → CNN expects (B, C, T)
        x = x.permute(0, 2, 1)            # (B, C, T)
        x = self.conv1(x)                 # (B, F, T_out)
        x = self.relu(x)
        x = self.pool(x)                  # (B, F, 1)
        x = self.flatten(x)               # (B, F)
        x = self.fc(x)                    # (B, output_dim)
        return x


In [ ]:
def train_cnn(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='Training CNN'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = torch.tensor(np.array(X_train)).float().to(DEVICE)  # (B, T, 1)
        y_train = torch.tensor(np.array(y_train)).float().to(DEVICE)  # (B, PREDICT)

        model = MultiOutputCNN(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]
        }

    return trained_models

In [ ]:
# 학습
trained_models = train_cnn(train)

Training CNN: 100%|██████████| 193/193 [07:28<00:00,  2.32s/it]


In [ ]:
def predict_cnn(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        if store_menu not in trained_models:
            continue

        model = trained_models[store_menu]['model']
        scaler = trained_models[store_menu]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        recent_vals = scaler.transform(recent_vals.reshape(-1, 1))
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)  # (1, T, 1)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()  # (PREDICT,)

        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)

In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('./test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_cnn(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/tmp/ipython-input-345547936.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x_input = torch.tensor([recent_vals]).float().to(DEVICE)  # (1, T, 1)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation

# 3. 1-D CNN 다층

In [ ]:
class DeepMultiOutputCNN(nn.Module):
    def __init__(self, input_dim=1, output_dim=7, num_filters=64, kernel_size=3):
        super(DeepMultiOutputCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=num_filters, kernel_size=kernel_size, padding=1)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=kernel_size, padding=1)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters, kernel_size=kernel_size, padding=1)
        self.relu3 = nn.ReLU()

        self.pool = nn.AdaptiveAvgPool1d(1)  # Output shape: (B, F, 1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(num_filters, output_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)         # (B, C, T)
        x = self.relu1(self.conv1(x))  # (B, F, T)
        x = self.relu2(self.conv2(x))  # (B, F, T)
        x = self.relu3(self.conv3(x))  # (B, F, T)
        x = self.pool(x)               # (B, F, 1)
        x = self.flatten(x)            # (B, F)
        return self.fc(x)              # (B, output_dim)


In [ ]:
def train_D_cnn(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='Training D_CNN'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = torch.tensor(np.array(X_train)).float().to(DEVICE)  # (B, T, 1)
        y_train = torch.tensor(np.array(y_train)).float().to(DEVICE)  # (B, PREDICT)

        model = DeepMultiOutputCNN(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]
        }

    return trained_models

In [ ]:
# 학습
trained_models = train_D_cnn(train)

Training D_CNN: 100%|██████████| 193/193 [10:32<00:00,  3.28s/it]


In [ ]:
def predict_D_cnn(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        if store_menu not in trained_models:
            continue

        model = trained_models[store_menu]['model']
        scaler = trained_models[store_menu]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        recent_vals = scaler.transform(recent_vals.reshape(-1, 1))
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)  # (1, T, 1)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()  # (PREDICT,)

        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)

In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('./test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_D_cnn(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/valida

# 4. Transformer

In [ ]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :].to(x.device)
        return x

class MultiOutputTransformer(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, output_dim=7):
        super(MultiOutputTransformer, self).__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, output_dim)

    def forward(self, x):
        # x: (B, T, 1)
        x = self.input_proj(x)  # (B, T, d_model)
        x = self.pos_encoder(x)  # (B, T, d_model)
        x = self.transformer_encoder(x)  # (B, T, d_model)
        return self.fc(x[:, -1, :])  # (B, output_dim)


In [ ]:
def train_transformer(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='Training Transformer'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = torch.tensor(np.array(X_train)).float().to(DEVICE)  # (B, T, 1)
        y_train = torch.tensor(np.array(y_train)).float().to(DEVICE)  # (B, PREDICT)

        model = MultiOutputTransformer(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (LOOKBACK, 1)
        }

    return trained_models


In [ ]:
# 학습
trained_models = train_transformer(train)

Training Transformer: 100%|██████████| 193/193 [30:11<00:00,  9.38s/it]


In [ ]:
def predict_transformer(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        if store_menu not in trained_models:
            continue

        model = trained_models[store_menu]['model']
        scaler = trained_models[store_menu]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals.reshape(-1, 1))  # (LOOKBACK, 1)
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)    # (1, LOOKBACK, 1)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()    # (PREDICT,)

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))  # 음수 방지

        # 예측일자 지정
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)


In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('./test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_transformer(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)


NameError: name 'predict_transformer' is not defined

# 변환부

In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # Sample submission의 모든 (영업일자, 메뉴명) 조합을 유지
    final_df = sample_submission.copy()
    pred_df['영업장명_메뉴명'] = pred_df['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

    # pred_df와 sample_submission을 merge
    # pred_df에 없는 조합은 NaN이 됨
    merged_df = pd.merge(
        final_df.melt(id_vars='영업일자', var_name='영업장명_메뉴명', value_name='sample_value'),
        pred_df,
        on=['영업일자', '영업장명_메뉴명'],
        how='left'
    )

    # 예측값이 없는 경우 0으로 채움
    merged_df['매출수량'] = merged_df['매출수량'].fillna(0)

    # 원래의 submission 형식으로 변환
    pivot_df = merged_df.pivot(index='영업일자', columns='영업장명_메뉴명', values='매출수량').reset_index()

    # 컬럼 순서를 sample_submission과 동일하게 맞춤
    final_columns = ['영업일자'] + [col for col in sample_submission.columns if col != '영업일자']
    # Pivot 시 생성되지 않은 컬럼 (예: 예측 대상이 아니었던 메뉴)은 0으로 채워 추가
    for col in final_columns:
        if col not in pivot_df.columns:
            pivot_df[col] = 0

    final_df = pivot_df[final_columns]

    return final_df

In [ ]:
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,6.715120
1,TEST_00+2일,느티나무 셀프BBQ_1인 수저세트,4.158187
2,TEST_00+3일,느티나무 셀프BBQ_1인 수저세트,2.642732
3,TEST_00+4일,느티나무 셀프BBQ_1인 수저세트,3.657989
4,TEST_00+5일,느티나무 셀프BBQ_1인 수저세트,5.607833
...,...,...,...
13505,TEST_09+3일,화담숲카페_현미뻥스크림,32.155990
13506,TEST_09+4일,화담숲카페_현미뻥스크림,33.247511
13507,TEST_09+5일,화담숲카페_현미뻥스크림,32.327479
13508,TEST_09+6일,화담숲카페_현미뻥스크림,27.694537


In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('baseline_submission.csv', index=False, encoding='utf-8-sig')

In [ ]:
submission

영업장명_메뉴명,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,6.715120,1.580249,8.384051,5.944964,0.370586,1.176424,5.147255,5.090495,2.655240,...,10.885117,28.738136,20.031957,6.002459,88.826232,61.403247,3.486834,64.848514,14.408350,26.030095
1,TEST_00+2일,4.158187,1.812146,3.379354,1.599559,0.164001,1.186541,8.509889,3.504929,2.095333,...,9.537480,21.102096,14.081488,6.730608,72.164953,49.399383,4.969262,56.969985,13.556120,17.180764
2,TEST_00+3일,2.642732,18.438915,2.375598,0.632615,0.248375,1.166676,12.298826,3.288429,1.959935,...,9.449302,19.279916,12.704040,4.950808,45.573147,46.109093,5.714893,54.195925,11.762493,11.231425
3,TEST_00+4일,3.657989,25.616419,2.517965,0.764107,0.395451,1.146787,12.118106,3.322363,2.224593,...,10.426984,21.972769,16.762409,6.117467,62.951599,45.055273,6.257522,53.232255,10.560454,11.269768
4,TEST_00+5일,5.607833,21.763279,4.150867,1.061172,0.492827,1.347157,10.862215,4.881844,2.603755,...,11.264660,21.714218,12.824125,6.152022,83.420027,44.101698,6.639820,54.117756,9.209793,17.867176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TEST_09+3일,3.361047,13.469928,1.687743,1.463906,0.000000,1.537833,9.076690,0.211745,1.966429,...,7.737284,11.777338,13.385885,4.670686,74.241973,30.559034,81.664517,137.993511,28.327485,32.155990
66,TEST_09+4일,3.267186,31.502464,3.789226,1.598540,0.179459,1.491635,11.768068,0.000000,2.121435,...,8.679214,13.656609,16.380877,6.176466,97.448572,30.908023,83.946528,132.466519,27.873162,33.247511
67,TEST_09+5일,2.381146,62.321299,4.732188,1.376629,0.000000,1.806510,14.781630,0.789231,2.374884,...,9.088421,13.477062,13.195573,5.233844,129.695730,25.905975,81.784250,122.624473,24.927588,32.327479
68,TEST_09+6일,3.602992,68.606330,6.223895,5.038282,0.924891,1.700192,15.738893,2.035862,2.915094,...,9.873111,15.734649,17.445688,7.591149,125.848559,30.520030,80.185249,118.739965,26.797960,27.694537
